In [1]:
import os
import pandas as pd
from pandas import read_csv
from pandas import DataFrame
from math import *
import math
from sympy import *
from sympy.abc import x,y,z,a,b,c,d
import numpy as np
import numpy
import csv
#F 

def preprocessing(before_filename, after_filename):
    train_dataset = read_csv(before_filename, header=0, index_col=None)
    train_values = train_dataset.values

    power = train_values[:, 0]
    temperature = train_values[:, 1]
    humidity = train_values[:, 2]
    speed = train_values[:, 3]
    common = [1 for i in range(power.shape[0])]

    transfer_power = []
    transfer_temperature = []
    transfer_humidity = []
    transfer_speed = []

    except_value = -5.0

    for index in range(len(power)):
        if round(log(power[index], e), 2) == -inf:
            transfer_power.append(except_value)
        else:
            transfer_power.append(round(log(power[index], e), 2))

    for index in range(len(temperature)):
        if round(log(temperature[index], e), 2) == -inf:
            transfer_temperature.append(except_value)
        else:
            transfer_temperature.append(round(log(temperature[index], e), 2))

    for index in range(len(humidity)):
        if round(log(humidity[index], e), 2) == -inf:
            transfer_humidity.append(except_value)
        else:
            transfer_humidity.append(round(log(humidity[index], e), 2))

    for index in range(len(speed)):
        if round(log(speed[index], e), 2) == -inf:
            transfer_speed.append(except_value)
        else:
            transfer_speed.append(round(log(speed[index], e), 2))

    transfer_common = numpy.array(common)

    Data = {'power': transfer_power, 'temperature': transfer_temperature, 'humidity': transfer_humidity,
            'speed': transfer_speed, 'common': transfer_common}
    df = DataFrame(Data, columns=['power', 'temperature', 'humidity', 'speed', 'common'])
    df.to_csv(after_filename, index=False)

#验证集的MAPE
def val_start(test_filename, n):
    dt = read_csv(test_filename, header=0, index_col=False)
    df = pd.DataFrame(dt)
    para = para_median(n)
    mape = 0
    #print(para)
    for index in range(len(df.values)):
        X = df.values[index][1:]
        testY = df.values[index][0]
        para = np.array(para).reshape(4, 1)
        predict = np.dot(X, para)
        mape = mape + abs(pow(e,testY)-pow(e,predict))/pow(e,testY)
   
    return mape/len(df.values)
   
#取前n个最优解参数的中位数
def para_median(n):
    lc = pd.DataFrame(pd.read_csv('Xe1_A.csv', header=None, names=['xulie', 'wucha']))#Xe1_A.csv,Xe1_B.csv...Xe1_F.csv
    tdd = lc.sort_values(by='wucha', ascending=True).head(n)
    index = list(tdd['xulie'])
    dt = pd.read_csv('Xee_A.csv', header=None, names=['xulie', '参数1', '参数2', '参数3', '参数4'])#Xee_A.csv,Xee_B.csv...Xee_F.csv
    df = pd.DataFrame(dt)
    X = []
    for i in index:
        X.append([df.values[i][1],df.values[i][2],df.values[i][3],df.values[i][4]])
    X = np.array(X)
    para1 = np.median(X[:, 0])
    para2 = np.median(X[:, 1])
    para3 = np.median(X[:, 2])
    para4 = np.median(X[:, 3])
    return [para1, para2, para3, para4]

#write the mape，accurary of LE_GRA to .csv file
def lr_mape_csv(filename3_test,n):
    mape = []
    for index in range(1, n):
        mape.append(val_start(filename3_test, index))

    min_index = mape.index(min(mape))
    #print(para_median(min_index))

    dt = read_csv(filename3_test, header=0, index_col=False)
    df = pd.DataFrame(dt)
    mape1 = 0
    #print(para)
    para = para_median(min_index + 1)
    predict_Y = []
    reality_Y = []
    mp = []
    acc = []
    for index in range(len(df.values)):
        X = df.values[index][1:]
        testY = df.values[index][0]
        reality_Y.append(pow(e,testY))
        para = np.array(para).reshape(4, 1)
        predict = np.dot(X, para)
        predict_Y.append(pow(e,predict))
        mape1 = mape1 + abs(pow(e,testY)-pow(e,predict))/pow(e,testY)
        mp.append(abs(pow(e,testY)-pow(e,predict))/pow(e,testY))
        acc.append(1-abs(pow(e,testY)-pow(e,predict))/pow(e,testY))
    Data = {'reality': reality_Y, 'predict': predict_Y, 'mape': mp, 'accurary': acc}
    df = DataFrame(Data, columns=["reality", "predict", "mape", "accurary"])
    df.to_csv("experimentA_LE_validation_result.csv", index=False)#experimentA_LE_validation_result.csv...experimentF_LE_validation_result.csv

#GRU模型的信息熵
def Entropy_GRU(filename3_gru_result):
    dt = read_csv(filename3_gru_result, header=0, index_col=False)
    df = pd.DataFrame(dt)

    mape = df.values[0:,2]
    n = len(mape)
    acc = []
    for index in range(n):
        acc.append(1-mape[index])

    P_GRU = []
    for index in range(n):
        P_GRU.append(acc[index]/sum(acc))
    P = 0
    for index in range(n):
        if P_GRU[index]<=0:
            continue
        else:
            P = P + P_GRU[index] * math.log(P_GRU[index],e)
    entropy_gru = - math.log(n,e) * P
    return entropy_gru

#LR模型的信息熵
def Entropy_LR(filename3_lr_result):
    dt = read_csv(filename3_lr_result, header=0, index_col=False)
    df = pd.DataFrame(dt)
    acc = df.values[0:, 3]
    n = len(acc)


    P_LR = []
    for index in range(n):
        P_LR.append(acc[index]/sum(acc))
    P = 0
    for index in range(n):
        if P_LR[index]<=0:
            continue
        else:
            P = P + P_LR[index] * math.log(P_LR[index],e)
    entropy_lr = -math.log(n,e) * P
    return entropy_lr

#GRU模型和LR模型分别的权重W_GRU,W_LR
def weight(filename3_gru_result,filename3_lr_result):
    entropy_gru = Entropy_GRU(filename3_gru_result)
    entropy_lr  = Entropy_LR(filename3_lr_result)
    W_GRU = (1 - entropy_gru)/(2 - (entropy_gru + entropy_lr))
    W_LR  = (1 - entropy_lr)/(2 -(entropy_gru + entropy_lr))
    return W_GRU, W_LR


acc_gru_lr = []

#混合模型的mape和准确率的方差
def ensemble_mape_var(filename3_gru_result,filename3_lr_result,filename3_reality,filename3_gru_r_result,filename3_lr_r_result):
    W_GRU,W_LR = weight(filename3_gru_result,filename3_lr_result)
    dt_lr = read_csv(filename3_lr_r_result, header=0, index_col=False)
    df_lr = pd.DataFrame(dt_lr)
    dt_gru = read_csv(filename3_gru_r_result, header=0, index_col=False)
    df_gru = pd.DataFrame(dt_gru)
    dt_test = read_csv(filename3_reality, header=0, index_col=False)
    df_test = pd.DataFrame(dt_test)
    n = len(df_test.values)
    mape = 0
    for index in range(n):
        predict = W_GRU * df_gru.values[index][1] + W_LR * df_lr.values[index][1]
        mape = mape + abs(df_test.values[index][0] - predict)/df_test.values[index][0]
        if abs(df_test.values[index][0] - predict)/df_test.values[index][0]<=1:
            acc_gru_lr.append(1-abs(df_test.values[index][0] - predict)/df_test.values[index][0])
    print("The mape of GRA_GRU is:", mape/n)
    print("The variance of GRA_GRU is:", np.var(acc_gru_lr))



#GRU模型准确率的方差
def gru_variance(filename3_gru_result):
    dt_gru = read_csv(filename3_gru_result, header=0, index_col=False)
    df_gru = pd.DataFrame(dt_gru)
    mape_gru = df_gru.values[0:, 2]
    n_gru = len(mape_gru)
    acc_gru = []
    for index in range(n_gru):
        if mape_gru[index] <= 1:
            acc_gru.append(1 - mape_gru[index])
    print("The variance of GRU is:", np.var(acc_gru))

#LR模型准确率的方差
def lr_variance(filename3_lr_result):
    dt_lr = read_csv(filename3_lr_result, header=0, index_col=False)
    df_lr = pd.DataFrame(dt_lr)
    mape_lr = df_lr.values[0:, 3]
    n_lr = len(mape_lr)
    acc_lr = []
    for index in range(n_lr):
        if mape_lr[index] >= 0:
            acc_lr.append(mape_lr)
    print("The variance of LE_GRA:", np.var(acc_lr))


if __name__ == "__main__":
#   preprocessing("validationA.csv", "valA.csv")#validationA.csv/valA.csv...validationF.csv/valF.csv
#   val_start("valA.csv",150)  #valA...valF
    lr_mape_csv("valA.csv", 150) #product experimentF_LE_validation_result.csv  #valA...valF
    ensemble_mape_var("experimentA_GRU_validation_result.csv", "experimentA_LE_validation_result.csv", "testingA.csv","experimentA_GRU_result.csv","experimentA_LE_result.csv")#"experimentA_GRU_validation_result.csv", "experimentA_LE_validation_result.csv", "testingA.csv","experimentA_GRU_result.csv","experimentA_LE_result.csv"..."experimentA_GRU_validation_result.csv", "experimentA_LE_validation_result.csv", "testingA.csv","experimentA_GRU_result.csv","experimentA_LE_result.csv"
    gru_variance("experimentA_GRU_result.csv")#experimentA_GRU_result.csv...experimentF_GRU_result.csv
    lr_variance("experimentA_LE_result.csv")#experimentA_LE_result.csv...experimentF_LE_result.csv

The mape of GRA_GRU is: 0.1353879100119654
The variance of GRA_GRU is: 0.008405628814192028
The variance of GRU is: 0.006925953144872238
The variance of LE_GRA: 0.01259354709982402
